## Twin Analysis

The HCP databases uses a twin sample, allowing for the comparison of monozygotic (MZ) vs dizygotic (DZ) twins. 

This allows us to compare the heritability of oscillations.

In [1]:
# Import required libraries/functions
from __future__ import print_function, division
import random
import itertools
import numpy as  np

# Import custom code from module om, including general functions and OO code for handling data
from om.core.db import OMDB
from om.core.osc import Osc
from om.core.io import load_meg_pairs
from om.core.utils import check_file_status
from om.meg.twin import *

from om.meg.twin import _comp_pf_pair, _comp_space_pair, _comp_osc_pair, _comp_sl_pair

# TODO: Make all this duplicated runs for 3 subj groups into functions.

In [2]:
# Pull out twin data from demographics file
mz_twins, dz_twins, all_twins, not_twins = get_twin_data()

# Match twin pairs
mz_id_pairs, mz_singles = match_twins(mz_twins)
dz_id_pairs, dz_singles = match_twins(dz_twins)

# Check how many pairs where extracted
print('There are', str(len(mz_id_pairs)), 'MZ twin pairs.')
print('There are', str(len(dz_id_pairs)), 'DZ twin pairs.')

There are 18 MZ twin pairs.
There are 19 DZ twin pairs.


In [3]:
# Get database object and set up database to use
db = OMDB()
dat_source = 'HCP'

# Check which twin subjects are available
av_dat, no_dat = check_file_status(all_twins, db, dat_source='HCP')

Of 93 subjects, 77 files are available and 16 files are unavailable.


In [4]:
# Given the missing data, check which twin pairs are complete
mz_complete_pairs = check_complete_pairs(mz_id_pairs, av_dat)
dz_complete_pairs = check_complete_pairs(dz_id_pairs, av_dat)

# Print out number of available pairs by twin type
print('There are', str(len(mz_complete_pairs)), 'complete MZ twin pairs.')
print('There are', str(len(dz_complete_pairs)), 'complete DZ twin pairs.')

There are 10 complete MZ twin pairs.
There are 16 complete DZ twin pairs.


In [5]:
# Check which pairs are missing (at least one) subject
print('MZ Twins Incomplete Pairs:')
print(list(set(mz_id_pairs) - set(mz_complete_pairs)))
print('DZ Twins Incomplete Pairs:')
print(list(set(dz_id_pairs) - set(dz_complete_pairs)))

MZ Twins Incomplete Pairs:
[(189349, 500222), (151526, 352132), (149741, 433839), (287248, 660951), (187547, 200109), (104012, 153732), (102816, 680957), (191033, 872764)]
DZ Twins Incomplete Pairs:
[(192641, 825048), (182840, 205119), (108323, 140117)]


In [6]:
# Get a list of all possible non-twin pairs, to compare to
all_pairs = list(itertools.combinations(av_dat, 2))
non_twin_pairs = rm_twin_pairs(all_pairs, mz_id_pairs + dz_id_pairs)

# Of all possible combinations, get a random sample of non-twin pairs, matching number of twin pairs
n_non_twin = len(dz_complete_pairs)
rand_inds = random.sample(range(len(non_twin_pairs)), n_non_twin)
non_twin_pairs = [non_twin_pairs[i] for i in rand_inds]

## Load Data

In [7]:
# Get database object and oscillatory band definition
db = OMDB()
osc = Osc(default=True)

In [8]:
# Load subjects into pairs
mz_twin_dat = load_meg_pairs(mz_complete_pairs, osc_bands_vert=True, osc=osc, db=db)
dz_twin_dat = load_meg_pairs(dz_complete_pairs, osc_bands_vert=True, osc=osc, db=db)
non_twin_dat = load_meg_pairs(non_twin_pairs, osc_bands_vert=True, osc=osc, db=db)

## Peak Frequencies

Compares the peak frequencies within oscillatory bands between twins and non-twins.

In [9]:
# Settings
peak_type = 'band'
avg = 'median'

# Compare peak frequencies
mz_peak_avg, mz_peak_dat = comp_peak_freq(mz_twin_dat, peak_type=peak_type, avg=avg)
dz_peak_avg, dz_peak_dat = comp_peak_freq(dz_twin_dat, peak_type=peak_type, avg=avg)
non_twin_peak_avg, non_twin_peak_dat = comp_peak_freq(non_twin_dat, peak_type=peak_type, avg=avg)

In [10]:
# Print out Results
print('MZ Twin Oscillatory Band Peak Frequency Results: ')
print_twin_results_vec(mz_peak_avg, osc.labels)
print('DZ Twin Oscillatory Band Peak Frequency Results: ')
print_twin_results_vec(dz_peak_avg, osc.labels)
print('Non-Twin Oscillatory Band Peak Frequency Results: ')
print_twin_results_vec(non_twin_peak_avg, osc.labels)

MZ Twin Oscillatory Band Peak Frequency Results: 
	 Theta 	 :  0.5708
	 Alpha 	 :  0.3523
	 Beta 	 :  2.0289
	 LowGamma 	 :  0.5651
DZ Twin Oscillatory Band Peak Frequency Results: 
	 Theta 	 :  1.0301
	 Alpha 	 :  0.7507
	 Beta 	 :  1.1748
	 LowGamma 	 :  1.1804
Non-Twin Oscillatory Band Peak Frequency Results: 
	 Theta 	 :  1.7826
	 Alpha 	 :  0.8616
	 Beta 	 :  1.8104
	 LowGamma 	 :  1.6822


### Oscillatory Band - Spatial Overlap

Compares the spatial topography of oscillatory bands between pairs of subject. 

In [11]:
# Compare spatial overlap
mz_space_avg, mz_space_dat = comp_osc_space(mz_twin_dat)
dz_space_avg, dz_space_dat = comp_osc_space(dz_twin_dat)
non_twin_space_avg, non_twin_space_dat = comp_osc_space(non_twin_dat)

In [12]:
# Print out Results
print('MZ Twin Oscillatory Band Spatial Overlap Results: ')
print_twin_results_vec(mz_space_avg, osc.labels)
print('DZ Twin Oscillatory Band Spatial Overlap Results: ')
print_twin_results_vec(dz_space_avg, osc.labels)
print('Non-Twin Oscillatory Band Spatial Overlap Results: ')
print_twin_results_vec(non_twin_space_avg, osc.labels)

MZ Twin Oscillatory Band Spatial Overlap Results: 
	 Theta 	 :  0.7723
	 Alpha 	 :  0.6776
	 Beta 	 :  0.9506
	 LowGamma 	 :  0.6287
DZ Twin Oscillatory Band Spatial Overlap Results: 
	 Theta 	 :  0.7723
	 Alpha 	 :  0.7059
	 Beta 	 :  0.9068
	 LowGamma 	 :  0.6073
Non-Twin Oscillatory Band Spatial Overlap Results: 
	 Theta 	 :  0.7332
	 Alpha 	 :  0.6695
	 Beta 	 :  0.9030
	 LowGamma 	 :  0.5958


### Oscillation Parameters Comparison

The following compares oscillatory parameters, such as center frequency, power and bandwidth from within oscillatory bands. 

NOTE: As implemented, the these comparisons are not specific to how subjects are similar/different. 
Subjects can differ in either:
- The spatial topography across which the oscillation band is found and/or
- The oscillatory parameter within the vertices where the oscillatory band is found

^ This can/will be parcelled out in further analyses:
- A basic check is simply to look at the degree of overlap of oscillatory band topographies between subjects. 

In [13]:
# Settings. Osc-Param: {0: CF, 1: Power, 2: BW}
osc_param = 0

# Compare oscillatory parameters
mz_param_avg, mz_param_dat = comp_osc_param(mz_twin_dat, osc_param)
dz_param_avg, dz_param_dat = comp_osc_param(dz_twin_dat, osc_param)
non_twin_param_avg, non_twin_param_dat = comp_osc_param(non_twin_dat, osc_param)

In [14]:
# Print out results
print('MZ Twin Oscillatory Band Center Frequency Results: ')
print_twin_results_corr(mz_param_avg, osc.labels)
print('DZ Twin Oscillatory Band Center Frequency Results: ')
print_twin_results_corr(dz_param_avg, osc.labels)
print('Non-Twin Oscillatory Band Center Frequency Results: ')
print_twin_results_corr(non_twin_param_avg, osc.labels)

MZ Twin Oscillatory Band Center Frequency Results: 
	 Theta 	 :  0.1485
	 Alpha 	 :  0.1337
	 Beta 	 :  0.1537
	 LowGamma 	 :  0.1201
DZ Twin Oscillatory Band Center Frequency Results: 
	 Theta 	 :  0.1404
	 Alpha 	 :  0.0734
	 Beta 	 :  0.1171
	 LowGamma 	 :  0.0860
Non-Twin Oscillatory Band Center Frequency Results: 
	 Theta 	 :  0.0855
	 Alpha 	 :  0.0025
	 Beta 	 :  0.0856
	 LowGamma 	 :  0.0437


## Slope Comparison

In [15]:
# Compare slopes
mz_slope_avg, mz_slope_dat = comp_slope(mz_twin_dat)
dz_slope_avg, dz_slope_dat = comp_slope(dz_twin_dat)
non_twin_slope_avg, non_twin_slope_dat = comp_slope(non_twin_dat)

In [16]:
# Print out results
print('MZ Twin Slope Results: ')
print_twin_results_corr(mz_slope_avg, ['Slope'])
print('DZ Twin Slope Results: ')
print_twin_results_corr(dz_slope_avg, ['Slope'])
print('Non-Twin Slope Results: ')
print_twin_results_corr(non_twin_slope_avg, ['Slope'])

MZ Twin Slope Results: 
	 Slope 	 :  0.3584
DZ Twin Slope Results: 
	 Slope 	 :  0.4772
Non-Twin Slope Results: 
	 Slope 	 :  0.3034


## TEST CODE

In [17]:
ind = 1
osc = Osc(default=True)

meg_subj_1 = MegData(db, 'HCP', osc)
meg_subj_2 = MegData(db, 'HCP', osc)

meg_subj_1.import_foof(mz_complete_pairs[ind][0])
meg_subj_1.osc_bands_vertex()
meg_subj_1.peak_freq('band')
meg_subj_2.import_foof(mz_complete_pairs[ind][1])
meg_subj_2.osc_bands_vertex()
meg_subj_2.peak_freq('band')

NameError: name 'MegData' is not defined